In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import sklearn
import scipy.stats
from random import randint
from sklearn import svm
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score,log_loss,auc
from sklearn import model_selection,linear_model,metrics

In [3]:
def pickle_data(filename, data):

    fileObject = open('PickleData/'+filename,'wb') 
    # this writes the object a to the
    # file named 'testfile'
    pickle.dump(data,fileObject) 

    # here we close the fileObject
    fileObject.close()
    
def load_pickle_data(filename):
    # we open the file for reading
    fileObject = open('PickleData/'+filename,'rb')  
    # load the object from the file into var b
    return pickle.load(fileObject)

## Load Data

In [4]:
df = load_pickle_data('normal_clean')

In [5]:
df_val = load_pickle_data('val_normal_clean')

In [8]:
X = df.drop('click',axis=1)
y = df['click']

In [9]:
X_val = df_val.drop('click',axis=1)
y_val = df_val['click']

## Undersample Training set

In [10]:
# Apply the random under-sampling
rus = RandomUnderSampler(return_indices=True)
X_resampled, y_resampled, idx_resampled = rus.fit_sample(X, y)
# X_res_vis = pca.transform(X_resampled)

In [13]:
clf = svm.SVC()
clf.fit(X_resampled, y_resampled)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [14]:
pred_y = clf.predict(X_val)

In [22]:
print('Log Loss:',log_loss(y_val, pred_y))

Log Loss: 11.908070433324431


In [24]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(y_val, pred_y))
print('RMSE:',rms)

RMSE: 0.5871677600555317


In [34]:
fpr, tpr, thresholds = metrics.roc_curve(y_val, pred_y, pos_label=1)
print('AUC:',round(metrics.auc(fpr, tpr),5))

AUC: 0.5653


In [36]:
print (metrics.classification_report(y_val, pred_y))
# print (metrics.accuracy_score(y, pred_y))

             precision    recall  f1-score   support

          0       1.00      0.66      0.79    303723
          1       0.00      0.48      0.00       202

avg / total       1.00      0.66      0.79    303925



In [38]:
from sklearn.metrics import classification_report,confusion_matrix

confusion_matrix = pd.DataFrame(
    confusion_matrix(y_val,pred_y), 
    columns=["Predicted False", "Predicted True"], 
    index=["Actual False", "Actual True"]
)
display(confusion_matrix)

,Predicted False,Predicted True
Actual False,199046,104677
Actual True,106,96
